In [ ]:
%matplotlib inline

In [ ]:
import requests
import json
import pandas as pd
import numpy as np

In [ ]:
auth_data = {
    'grant_type'    : 'client_credentials',
    'client_id'     : '4840067a72b940a0bfb573681b84d320',
    'client_secret' : 'cff9770d549da86d5a6f539ac1df26fe03209610faa03ecbd57f0114fafc0389',
    'scope'         : 'read_product_data read_financial_data read_content'
}

# create Session instance
session = requests.Session()

# make a POST to retrieve access_token
auth_request = session.post('https://idfs.gs.com/as/token.oauth2', data = auth_data)
access_token_dict = json.loads(auth_request.text)
access_token = access_token_dict['access_token']

# update session headers
session.headers.update({'Authorization':'Bearer '+ access_token})

print(access_token)

In [ ]:
request_url = 'https://api.marquee.gs.com/v1/data/catalog/USCANFPP_MINI'
request = session.get(url=request_url)
data = json.loads(request.text)
print(data['fields'])

In [110]:
import pandas as pd

def get_ids(num_ids):
    offset = 0
    gsids = []
    while len(gsids) < num_ids:
        request_url = 'https://api.marquee.gs.com/v1/data/USCANFPP_MINI/coverage?limit=100&offset={}'.format(offset)
        request = session.get(url=request_url)
        data = json.loads(request.text)
        df = pd.DataFrame(data['results'])
        if len(df) == 0:
            print('Reached limit {}'.format(len(gsids)))
            break
        offset += 100
        gsids += list(df.gsid)
        print('Saved', offset, 'ids')
    # Check that we aren't getting duplicates
    return gsids

gsids = get_ids(100000)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [116]:
def get_names(gsids):
    data = []
    for i in range(len(gsids) // 100):
        print(i)
        payload = {
            "where": {
                "gsid": gsids[100 * i: 100 * i + 100]
            },
            "fields": [ "gsid", "ticker", "name" ],
            "limit": 100
        }

        request_url = 'https://api.marquee.gs.com/v1/assets/data/query'
        request = session.post(url=request_url, json = payload)
        results = json.loads(request.text)
        data += results['results']
    return data
    
data = get_names(gsids)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [117]:
company_df = pd.DataFrame(data)
company_df.to_csv('../data/processed/gs_companies_100k.csv', index=False)

In [ ]:
payload = {
    "startDate": "2015-11-02",
    "endDate": "2015-11-02",
    "where": {
        "gsid": list(gsids)
    }
}

request_url = 'https://api.marquee.gs.com/v1/data/USCANFPP_MINI/query'
request = session.post(url=request_url, json = payload)
results = json.loads(request.text)
print(results)
data = results['data']
print(data)
